In [1]:
import tensorflow as tf
import torch

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
df_raw=pd.read_csv('../raw_data/pima-indians-diabetes.data.csv')
df = df_raw.copy()

In [11]:
# pps
from sklearn.preprocessing import StandardScaler

ssc = StandardScaler()
df.iloc[:,:-1]=ssc.fit_transform(df.iloc[:,:-1])

### Modeling
```
 1. tf_low
 2. tf_high
 3. torch_low
 4. torch_high
 ```

In [12]:
x = tf.constant(df.iloc[:,:-1].values, tf.float32)
y = tf.constant(df.iloc[:,-1].values.reshape(-1,1), tf.float32)

initX = tf.initializers.GlorotUniform()
w = tf.Variable(initX(shape=[x.shape[-1],1]))
b = tf.Variable(initX(shape=[1]))

# w = tf.Variable([8,1],dtype=tf.float32)
# b = tf.Variable([1],tf.float32,dtype=tf.float32)

def cost():
    z = tf.matmul(x,w) + b
    c_i = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=z)
    c = tf.reduce_mean(c_i)
    return c

def hxFn(xdata):
    xd = tf.constant(xdata,tf.float32)
    z = tf.matmul(xd,w) + b
    hx = tf.nn.sigmoid(z)
    return (hx.numpy()>0.5) + 0

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)

for i in range(1000):
    optimizer.minimize(cost,var_list=[w,b])
    acc = accuracy_score(hxFn(df.iloc[:,:-1]),df.iloc[:,-1])
    print(f'epoch => {i} / cost => {cost().numpy()} / acc => {acc}')

/opt/homebrew/Caskroom/miniforge/base/envs/deep_learning/lib/python3.9/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


epoch => 0 / cost => 1.102451205253601 / acc => 0.4348958333333333
epoch => 1 / cost => 1.0138790607452393 / acc => 0.48046875
epoch => 2 / cost => 0.9387614130973816 / acc => 0.5260416666666666
epoch => 3 / cost => 0.8734239935874939 / acc => 0.56640625
epoch => 4 / cost => 0.8166378736495972 / acc => 0.6080729166666666
epoch => 5 / cost => 0.7670513391494751 / acc => 0.6328125
epoch => 6 / cost => 0.7233065366744995 / acc => 0.6471354166666666
epoch => 7 / cost => 0.6844336986541748 / acc => 0.6549479166666666
epoch => 8 / cost => 0.6500274538993835 / acc => 0.6731770833333334
epoch => 9 / cost => 0.6199700236320496 / acc => 0.6861979166666666
epoch => 10 / cost => 0.593953013420105 / acc => 0.6953125
epoch => 11 / cost => 0.5713116526603699 / acc => 0.7083333333333334
epoch => 12 / cost => 0.5513896942138672 / acc => 0.7252604166666666
epoch => 13 / cost => 0.5339574217796326 / acc => 0.7317708333333334
epoch => 14 / cost => 0.5192078351974487 / acc => 0.7421875
epoch => 15 / cost =

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model=Sequential()
model.add(Dense(units=1,
                input_dim=df.iloc[:,:-1].shape[-1], 
                activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1),
              metrics=['acc'])
hist = model.fit(df.iloc[:,:-1],df.iloc[:,-1],epochs=50)

Epoch 1/50
24/24 [==============================] - 0s 4ms/step - loss: 0.5372 - acc: 0.7344
Epoch 2/50
 1/24 [>.............................] - ETA: 0s - loss: 0.6704 - acc: 0.6875

2022-12-01 14:47:06.791610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 0s 4ms/step - loss: 0.4956 - acc: 0.7708
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4876 - acc: 0.7682
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4837 - acc: 0.7721
Epoch 5/50
24/24 [==============================] - 0s 3ms/step - loss: 0.4801 - acc: 0.7708
Epoch 6/50
24/24 [==============================] - 0s 3ms/step - loss: 0.4821 - acc: 0.7839
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4821 - acc: 0.7669
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4890 - acc: 0.7656
Epoch 9/50
24/24 [==============================] - 0s 3ms/step - loss: 0.4863 - acc: 0.7852
Epoch 10/50
24/24 [==============================] - 0s 3ms/step - loss: 0.4913 - acc: 0.7643
Epoch 11/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4836 - acc: 0.7695
Epoch 12/50
24/24 [==============================] - 0s 4ms/step - loss: 0.4898

In [14]:
# 3
from torch.nn import functional as F

x = torch.FloatTensor(df.iloc[:,:-1].values)
y = torch.FloatTensor(df.iloc[:,-1].values.reshape(-1,1))

w = torch.empty([x.shape[-1],1],requires_grad=True)
b = torch.empty([1],requires_grad=True)
torch.nn.init.uniform_(w)
torch.nn.init.uniform_(b)

def cost():
    z = torch.matmul(x,w) + b
    c_i = F.binary_cross_entropy_with_logits(z,y)
    c = torch.mean(c_i)
    return c

def hxFn(xdata):
    xd = torch.FloatTensor(xdata)
    z = torch.matmul(xd,w) + b
    hx = torch.sigmoid(z)
    return (hx.detach().numpy() > 0.5) + 0

optimizer = torch.optim.Adam([w,b],lr=0.1)

for i in range(1000):
    c=cost()
    optimizer.zero_grad()
    c.backward()
    optimizer.step()
    acc = accuracy_score(df.iloc[:,-1],hxFn(x))
    print(f'epoch => {i} / cost => {cost().item()} acc => {acc} ')

epoch => 0 / cost => 0.7786932587623596 acc => 0.6302083333333334 
epoch => 1 / cost => 0.7112100720405579 acc => 0.6432291666666666 
epoch => 2 / cost => 0.6555108428001404 acc => 0.6614583333333334 
epoch => 3 / cost => 0.6130313277244568 acc => 0.69140625 
epoch => 4 / cost => 0.5826436281204224 acc => 0.7057291666666666 
epoch => 5 / cost => 0.5605625510215759 acc => 0.71484375 
epoch => 6 / cost => 0.5429816842079163 acc => 0.7278645833333334 
epoch => 7 / cost => 0.527972400188446 acc => 0.7330729166666666 
epoch => 8 / cost => 0.5152062773704529 acc => 0.75 
epoch => 9 / cost => 0.504879891872406 acc => 0.7591145833333334 
epoch => 10 / cost => 0.4970422685146332 acc => 0.76171875 
epoch => 11 / cost => 0.4914382994174957 acc => 0.7591145833333334 
epoch => 12 / cost => 0.48760852217674255 acc => 0.7643229166666666 
epoch => 13 / cost => 0.4850383996963501 acc => 0.7643229166666666 
epoch => 14 / cost => 0.48326873779296875 acc => 0.7643229166666666 
epoch => 15 / cost => 0.4819

In [15]:
#4
import torch
from torch.optim import Adam
from torch.nn import BCELoss # Binary Cross Entropy


x = torch.FloatTensor(df.iloc[:,:-1].values)
y = torch.FloatTensor(df.iloc[:,-1].values.reshape(-1,1))

model =torch.nn.Sequential()
model.add_module('nn1',torch.nn.Linear(x.shape[-1],1))
model.add_module('sig1',torch.nn.Sigmoid())
loss_fn = BCELoss()
optimizer = Adam( model.parameters(), lr=0.1)

for epoch in range(1000):
    optimizer.zero_grad()
    hx = model(x)
    """
    => model.forward(x)
    => z = matmul(x,w) + b : Linear
    => Sigmoid(z) : Sigmoid
    """
    c = loss_fn(hx,y)
    c.backward()
    optimizer.step()
    pred=(hx.detach().numpy() > 0.5) +0
    print(f'epoch => {epoch} / cost => {c.detach().numpy()} acc => {accuracy_score(pred,y)} ')

epoch => 0 / cost => 0.797255277633667 acc => 0.37890625 
epoch => 1 / cost => 0.7045090794563293 acc => 0.546875 
epoch => 2 / cost => 0.6408748030662537 acc => 0.65234375 
epoch => 3 / cost => 0.5989684462547302 acc => 0.6927083333333334 
epoch => 4 / cost => 0.5697024464607239 acc => 0.7057291666666666 
epoch => 5 / cost => 0.5481541156768799 acc => 0.7174479166666666 
epoch => 6 / cost => 0.5316913723945618 acc => 0.7291666666666666 
epoch => 7 / cost => 0.5189791321754456 acc => 0.73046875 
epoch => 8 / cost => 0.5094032883644104 acc => 0.7408854166666666 
epoch => 9 / cost => 0.5024351477622986 acc => 0.74609375 
epoch => 10 / cost => 0.4975064694881439 acc => 0.7526041666666666 
epoch => 11 / cost => 0.49407675862312317 acc => 0.7552083333333334 
epoch => 12 / cost => 0.4917178452014923 acc => 0.7708333333333334 
epoch => 13 / cost => 0.49012264609336853 acc => 0.7734375 
epoch => 14 / cost => 0.48905205726623535 acc => 0.76953125 
epoch => 15 / cost => 0.48828569054603577 acc =